In [1]:
import os
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Download the Postgres driver for Spark
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 88.7 kB in 4s (22.4 kB/s)
Reading package lists... Done
--2022-

In [2]:
# Set up Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Amazon-Reviews-Analysis").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Dataset into Spark DataFrame

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Beauty_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Beauty_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

df.printSchema()

df.show()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+---------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rat

### Create DataFrames to match database tables

In [4]:
from pyspark.sql.functions import to_date

In [5]:
# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count")

customers_df.printSchema()

customers_df.show()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|    9015608|             3|
|    9767880|             2|
|   15958313|             1|
|   12879980|            20|
|   43920023|            31|
|     106535|             1|
|   20328662|             1|
|   12819130|             4|
|   50145309|             1|
|   15159322|             2|
|   16747249|             2|
|   37678162|             1|
|     227763|             1|
|   15191144|             2|
|   31946913|             1|
|   44295916|             1|
|   19021463|             1|
|     806337|             2|
|   16306618|            18|
|     393186|             1|
+-----------+--------------+
only showing top 20 rows



In [14]:
# Create the products_table DataFrame and drop duplicates 
products_df = df.select(["product_id", "product_title"]).drop_duplicates(['product_id'])

products_df.printSchema()

products_df.show()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|1935682016|Henna Art, Tools ...|
|9788077757|Overnight Success...|
|9790770367|Dolce & Gabbana C...|
|9790773587|Calvin Klein Euph...|
|9790796927|Herve Leger Perfu...|
|9790800886|Trouble By Bouche...|
|9865962039|Women Perfume Oil...|
|B000050B6Q|Conair 077B 1875-...|
|B000052YQV|Pond's Overnight ...|
|B000052ZBD|Neutrogena Rainba...|
|B000052ZEO|Wet 'n' Wild Crem...|
|B0000532VT|Neutrogena Clean ...|
|B00005CDRS|Sunbeam SBCL870 1...|
|B00005NAPK|Cover Girl Outlas...|
|B00006ANDK|Oral-B Sensitive ...|
|B000089SAM|AXE Deodorant Bod...|
|B00008BFTP|Philips HQ481 Tra...|
|B00008KA86|Revlon ColorStay ...|
|B0000A4EW2|Aveeno Moisture L...|
|B0000AKT6Q|Therapist Select ...|
+----------+--------------------+
only showing top 20 rows



In [7]:
# Create the review_id_table DataFrame
# Convert the 'review_date' column to 'date' datatype
review_id_df = df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])

review_id_df.printSchema()

review_id_df.show()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3I2DHQBR577SS|    1797882|B001ANOOOE|       2102612| 2015-08-31|
|R1QNE9NQFJC2Y4|   18381298|B0016J22EQ|     106393691| 2015-08-31|
|R3LIDG2Q4LJBAO|   19242472|B00HU6UQAG|     375449471| 2015-08-31|
|R3KSZHPAEVPEAL|   19551372|B002HWS7RM|     255651889| 2015-08-31|
| RAI2OIG50KZ43|   14802407|B00SM99KWU|     116158747| 2015-08-31|
|R1R30FA4RB5P54|    2909389|B000NYL1Z6|     166146615| 2015-08-31|
|R30IJKCGJBGPJH|   19397215|B001SYWTFG|     111742328| 2015-08-31|
|R18GLJJPVQ1OVH|    3195210|B005F2EVMQ|     255803087| 2015-08-31|
| R8TVYIJXLYJT0|   52216383|B00M1SUW7K|   

In [23]:
# Create the vine_table DataFrame
vine_df = df.select(["review_id", ("star_rating"), "helpful_votes", "total_votes", "vine", "verified_purchase"])\

# Recast star_rating to integer type
vine_df = vine_df.withColumn("star_rating", vine_df["star_rating"].cast('int'))

vine_df.printSchema()

vine_df.show()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3I2DHQBR577SS|          5|            0|          0|   N|                Y|
|R1QNE9NQFJC2Y4|          5|            0|          0|   N|                Y|
|R3LIDG2Q4LJBAO|          5|            0|          0|   N|                Y|
|R3KSZHPAEVPEAL|          5|            0|          0|   N|                Y|
| RAI2OIG50KZ43|          5|            0|          0|   N|                Y|
|R1R30FA4RB5P54|          4|            0|          0|   N|                Y|
|R30IJKCGJBGPJH|          5|    

### Connect to the AWS RDS instance and write each DataFrame to its table 

In [9]:
# Prompt user for password
from getpass import getpass
password = getpass('Enter database password')

# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://amazon-vine-analysis.ciotkzypfeyx.us-east-1.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [11]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [15]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [16]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [24]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)